In [1]:
# Załadowanie pakietów
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsRegressor, RadiusNeighborsRegressor
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score

In [2]:
df = pd.read_csv('/home/pawcio/Desktop/Projekty/jdsz4-dept/projekt_ml/dataset/insurance_hot_version.csv')

In [3]:
df.head(10)

,age,bmi,children,charges,sex_female,sex_male,smoker_no,smoker_yes,region_northeast,region_northwest,region_southeast,region_southwest
0,19,27.900,0,16884.92400,1,0,0,1,0,0,0,1
1,18,33.770,1,1725.55230,0,1,1,0,0,0,1,0
2,28,33.000,3,4449.46200,0,1,1,0,0,0,1,0
3,33,22.705,0,21984.47061,0,1,1,0,0,1,0,0
4,32,28.880,0,3866.85520,0,1,1,0,0,1,0,0
5,31,25.740,0,3756.62160,1,0,1,0,0,0,1,0
6,46,33.440,1,8240.58960,1,0,1,0,0,0,1,0
7,37,27.740,3,7281.50560,1,0,1,0,0,1,0,0
8,37,29.830,2,6406.41070,0,1,1,0,1,0,0,0
9,60,25.840,0,28923.13692,1,0,1,0,0,1,0,0


In [4]:
df.max()

age                    64.00000
bmi                    53.13000
children                5.00000
charges             63770.42801
sex_female              1.00000
sex_male                1.00000
smoker_no               1.00000
smoker_yes              1.00000
region_northeast        1.00000
region_northwest        1.00000
region_southeast        1.00000
region_southwest        1.00000
dtype: float64

In [5]:
x = df[['age', 'bmi', 'children', 'sex_female', 'sex_male',
       'smoker_no', 'smoker_yes', 'region_northeast', 'region_northwest',
       'region_southeast', 'region_southwest']]
y = df['charges']

In [16]:
new_x = np.array(x)
new_x

array([[19.  , 27.9 ,  0.  , ...,  0.  ,  0.  ,  1.  ],
       [18.  , 33.77,  1.  , ...,  0.  ,  1.  ,  0.  ],
       [28.  , 33.  ,  3.  , ...,  0.  ,  1.  ,  0.  ],
       ...,
       [18.  , 36.85,  0.  , ...,  0.  ,  1.  ,  0.  ],
       [21.  , 25.8 ,  0.  , ...,  0.  ,  0.  ,  1.  ],
       [61.  , 29.07,  0.  , ...,  1.  ,  0.  ,  0.  ]])

In [12]:
def kNN_function(x, y):
    
    print('---------------------\n')
    print('kNN regression: \n')
    
    # Podział dataset
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.33, random_state = 42)
    
    # Przedstawienie liczebności zbiorów
    print(' Liczebność zbioru T = ', x_train.count()[0],'\n','Liczebność zbioru V = ', x_test.count()[0])
    print('---------------------\n')
    
    # Standaryzacja wartości x
    scaler = MinMaxScaler()
    scaler.fit(x_train)
    norm_x_train = scaler.transform(x_train)
    norm_x_test = scaler.transform(x_test)
    
    # Wytrenowanie modelu
    model = KNeighborsRegressor()
    model_fit = model.fit(norm_x_train, y_train)
    
    # Obliczenie wartości y_pred
    y_pred = model_fit.predict(norm_x_test)
    r2 = r2_score(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    
    print(' Wartość współczynnika R^2 = ', r2)
    print(' Wartość współczynnika mse = ', mse)
    print('---------------------\n')
    # Znalezienie najlepszych hiperparametrów

    print(' Najlepszsze parametry modelu: \n')
    
        
    params = {'n_neighbors': range(1,20,1),
              'algorithm': ('ball_tree', 'kd_tree'),
              'leaf_size': range(1,21, 2),
              'p': range(1,3,1)}
    
    # Najlepsze parametry dla modelu
    kNN_regressor_search = GridSearchCV(model, params, cv = 5)
    kNN_regressor_search.fit(norm_x_train, y_train)
    best_params = kNN_regressor_search.best_params_
    print(best_params)
    
    best_model = \
        KNeighborsRegressor(n_neighbors= best_params['n_neighbors'], 
                            algorithm = best_params['algorithm'],
                            leaf_size=best_params['leaf_size'], 
                            p=best_params['p'])
    best_model.fit(norm_x_train, y_train)
    bp_sc = best_model.score(norm_x_train, y_train)
    
    print('---------------------\n')
    print('Score = ', bp_sc)
    return best_model

In [14]:
best_model = kNN_function(x,y)

---------------------

kNN regression: 

 Liczebność zbioru T =  896 
 Liczebność zbioru V =  442
---------------------

 Wartość współczynnika R^2 =  0.7655099662907512
 Wartość współczynnika mse =  34363261.304650225
---------------------

 Najlepszsze parametry modelu: 

{'algorithm': 'ball_tree', 'leaf_size': 1, 'n_neighbors': 4, 'p': 1}
---------------------

Score =  0.8540968922587548


In [15]:
from joblib import dump, load
dump(best_model, 'model.joblib')

['model.joblib']